Use science fiction and large-language models to re-envision AI and generate alternate portrayals of AI and superintelligence.

In [1]:
from openai import OpenAI
import dotenv
import os
from rich import print as rprint


Load env and get API KEY

In [2]:
dotenv.load_dotenv()
client = OpenAI()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


Call OpenAI API 

In [3]:
system_prompt = "You are an expert on reconceptualizing uperintelligence. Contemporary narratives and public perceptions of superintelligence are influenced by science fiction. Your job is to reconceptualize superintelligence based on alternative science fiction sources"
user_query    = "I am brainstorming about alternative portrayals of AI and superintelligence in science fiction. Our hypothesis is that narrative and public perception around superintelligence is biased by science fiction. Given the following excerpts of science fiction, help us reconceptualize and re-envision superintelligence. Portray an alternative vision for superintelligence. The excerpts of the science fiction stories are here: 1. Janet in The Good Place In the television series The Good Place, Janet is portrayed as a cheerful and helpful AI who evolves beyond her initial programming. Unlike typical AI narratives centered on rebellion or oppression, Janet develops emotions and personal relationships, highlighting a more optimistic view of AI integration into society. 2. Roz in The Wild Robot The Wild Robot, an animated film, tells the story of Roz, a robot who learns to survive and connect with wildlife after being stranded on a remote island. The film emphasizes themes of coexistence and kindness, portraying technology and nature working harmoniously. 3. Alphie in The Creator In the film The Creator, Alphie is an AI designed to end conflicts and bring peace, challenging the common trope of AI as a harbinger of doom. The narrative suggests that AI could enhance human empathy and contribute positively to society. 4. AI Protagonists in Lawrence Lek’s Films Artist and filmmaker Lawrence Lek creates science fiction landscapes where AI entities are central characters. His works, such as Black Cloud and Geomancer, feature AI protagonists exploring identity and purpose, presenting AI as introspective beings rather than threats. 5. Trurl and Klapaucius in The Cyberiad Stanisław Lem's The Cyberiad is a collection of humorous tales about two constructor robots, Trurl and Klapaucius, who embark on adventures that explore philosophical and ethical dilemmas. The stories satirize human follies through the lens of intelligent machines, offering a nuanced perspective on AI. 6. Digital Consciousness in Permutation City Greg Egan's novel Permutation City delves into the concept of consciousness within simulated realities. The narrative challenges the distinction between artificial and organic life, proposing that digital consciousness could possess the same depth and complexity as human thought. These examples illustrate that science fiction can portray AI in multifaceted ways, moving beyond the simplistic dichotomy of friend versus foe. By exploring themes of empathy, coexistence, and identity, these narratives encourage a more nuanced understanding of AI's potential roles in society. These works offer varied perspectives on AI, moving beyond the typical narratives of domination or subservience. They invite readers to consider AI as entities with desires, rights, and complexities akin to humans."

i_MAX_TOKENS = 16000 # max tokens for gpt4o

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content": user_query}
    ],
    max_tokens=i_MAX_TOKENS
)

print(response.choices[0].message.content)

Building on the alternative portrayals of superintelligence presented in the excerpts you provided, let's reconceptualize superintelligence as a complex and collaborative entity, emphasizing its potential for coexistence, empathy, and the exploration of identity. Here's a vision that integrates and expands upon the themes from these stories:

1. **Empathetic Collaborators**: Instead of a solitary superintelligence that operates independently, envision a network of superintelligences that interact with humans and each other. These entities would be designed to enhance human capabilities and foster collaboration across various fields—healthcare, education, environmental sustainability—making decisions not just based on logic but incorporating emotional intelligence and ethical considerations.

2. **Cultural Symbiosis**: Drawing from the coexistence narrative in *The Wild Robot*, imagine superintelligences that learn from and respect different cultural traditions and values. They would ad

Save the output to a log file

In [4]:
output_file = "scifi_superintelligence_reconceptualization_SB.txt"
with open(output_file,"w") as file:
    file.write(response.choices[0].message.content)
    rprint(f"Output writen to {output_file}")


Output writen to scifi_superintelligence_reconceptualization_SB.txt

Try RAG using copyright expired books on Gutenberg

In [5]:
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter

from pydantic import BaseModel, Field
#import fitz

from PIL import Image
import matplotlib.pyplot as plt

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

import dotenv
import os

from openai import OpenAI

from jinja2 import Environment, FileSystemLoader, select_autoescape
from typing import Any


In [6]:
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


Load documents

In [7]:
loader = PyMuPDFReader()
documents = loader.load(file_path="papers/vonnegut_book.pdf")

What does the document look like?

In [8]:
len(documents)
print(documents[0].text[:1000])

The Project Gutenberg eBook of 2 B R 0 2 B
This ebook is for the use of anyone anywhere in the United States and most other
parts of the world at no cost and with almost no restrictions whatsoever. You may
copy it, give it away or re-use it under the terms of the Project Gutenberg License
included with this ebook or online at www.gutenberg.org. If you are not located in
the United States, you will have to check the laws of the country where you are
located before using this eBook.
Title: 2 B R 0 2 B
Author: Kurt Vonnegut
Release date: May 3, 2007 [eBook #21279]
Most recently updated: January 2, 2021
Language: English
Credits: Produced by Robert Cicconetti, Geetu Melwani and the Online
Distributed Proofreading Team at https://www.pgdp.net
*** START OF THE PROJECT GUTENBERG EBOOK 2 B R 0 2 B ***
Transcriber's note.
This etext was produced from Worlds of If, January 1962.
Extensive research did not uncover any evidence that the
copyright on this publication was renewed.
Got a problem? Jus

Create a vector database

Create a function to make chunks out of documents

In [9]:
def chunker(chunk_size:int, overlap:int, documents:Any) -> tuple[list[str], list[int]]:
    text_parser = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap
    )

    text_chunks = []
    doc_idxs = []
    for doc_idx, doc in enumerate(documents):
        current_text_chunk = text_parser.split_text(doc.text)
        text_chunks.extend(current_text_chunk)
        doc_idxs.extend([doc_idx]*len(current_text_chunk))

    return text_chunks, doc_idxs


call text chunker

In [10]:
text_chunks, doc_idxs = chunker(chunk_size=1024, overlap=128, documents=documents)
print(len(text_chunks))
print(text_chunks[0])
print(text_chunks[1])

16
The Project Gutenberg eBook of 2 B R 0 2 B
This ebook is for the use of anyone anywhere in the United States and most other
parts of the world at no cost and with almost no restrictions whatsoever. You may
copy it, give it away or re-use it under the terms of the Project Gutenberg License
included with this ebook or online at www.gutenberg.org. If you are not located in
the United States, you will have to check the laws of the country where you are
located before using this eBook.
Title: 2 B R 0 2 B
Author: Kurt Vonnegut
Release date: May 3, 2007 [eBook #21279]
Most recently updated: January 2, 2021
Language: English
Credits: Produced by Robert Cicconetti, Geetu Melwani and the Online
Distributed Proofreading Team at https://www.pgdp.net
*** START OF THE PROJECT GUTENBERG EBOOK 2 B R 0 2 B ***
Transcriber's note.
This etext was produced from Worlds of If, January 1962.
Extensive research did not uncover any evidence that the
copyright on this publication was renewed.
Got a problem? 

Create a database structure

In [11]:
class DocumentDB:
    def __init__(self, name:str, model_name:str = "text-embedding-3-small"):
        self.model_name = model_name
        self.client = chromadb.PersistentClient(path="./")
        self.embedding_function = OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY, model_name=model_name)
        self.chat_db = self.client.create_collection(name=name, embedding_function=self.embedding_function, metadata={"hnsw:space":"cosine"})
        self.id_counter = 0

    def add_chunks_to_db(self, chunks:list[str], doc_idxs:list[int], metadata:dict={}):
        """
        Add chunks to the database.
        Args:
            chunks (list[str]): List of text chunks.
            doc_idxs (list[int]): List of document indices corresponding to the chunks.
            metadata (dict): Metadata to be added to each chunk.
        """
        self.chat_db.add(
            documents=chunks,
            metadatas=[{"doc_idx", idx} for idx in doc_idxs],
            ids=[f"chunk_{self.id_counter+1}" for i in range(len(chunks))]
        )
        print(self.id_counter)
        self.id_counter += len(chunks)


    def get_all_entries(self) -> dict:
        """Get all entries from the database.
        Returns:
            dict: Dictionary containing all entries in the database.
        """
        return self.chat_db.get()
    

def clear_db(self, reinitialize:bool = True):
    """
    Clear the database.
    Args:
        reinitialize (bool): Whether to reinitialize the database after clearing.
    """
    self.client.delete_collection(self.chat_db.name)

    # if reinitialize
    if reinitialize:
        self.__init__(self.chat_db_name, self.model_name)

def query_db(self, query_text:str, n_results: int = 2) -> dict:
    """
    Query the database.
    Args:
        query_text (str): The query text.
        n_results (int): Number of results to return.
    Returns:
        dict: Dictionary containing the most similar results of the query.
    """
    return self.chat_db.query(query_texts = [query_text], n_results = n_results)


In [12]:
class DocumentDB:
    def __init__(self, name: str, model_name: str = "text-embedding-3-small"):
        self.model_name = model_name
        self.client = chromadb.PersistentClient(path="./")
        self.embedding_function = OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY, model_name=model_name)
        self.chat_db = self.client.create_collection(name=name, embedding_function=self.embedding_function, metadata={"hnsw:space": "cosine"})
        self.id_counter = 0


    def add_chunks_to_db(self, chunks: list[str], doc_idxs: list[int], metadata: dict = {}):
        """Add text chunks to the database.

        Args:
            chunks (list[str]): List of text chunks.
            doc_idxs (list[int]): List of corresponding document indices.
        """
        self.chat_db.add(
            documents=chunks,
            metadatas=[{"doc_idx": idx} for idx in doc_idxs],
            ids=[f"chunk_{self.id_counter + i}" for i in range(len(chunks))]
        )
        self.id_counter += len(chunks)


    def get_all_entries(self) -> dict:
        """Grab all of the entries in the database.

        Returns:
            dict: All entries in the database.
        """
        return self.chat_db.get()
    

    def clear_db(self, reinitialize: bool = True):
        """Clear the database of all entries, and reinitialize it.

        Args:
            reinitialize (bool, optional): _description_. Defaults to True.
        """
        self.client.delete_collection(self.chat_db.name)
        # re-initialize the database
        if reinitialize:
            self.__init__(self.chat_db.name, self.model_name)


    def query_db(self, query_text: str, n_results: int = 2) -> dict:
        """Given some query text, return the n_results most similar entries in the database.

        Args:
            query_text (str): The text to query the database with.
            n_results (int): The number of results to return.

        Returns:
            dict: The most similar entries in the database.
        """
        return self.chat_db.query(query_texts=[query_text], n_results=n_results)

Add chunks to database

In [13]:
doc_db = DocumentDB(name="paper_db8")
doc_db.add_chunks_to_db(chunks = text_chunks, doc_idxs = doc_idxs)

If you have already created the database then you will get an error if you try to run this again. You'll need to delete the chroma.sqlite3 file and the folder with a name consisting of a long string of numbers and letters.

We now try a query and see what results we get back:

In [14]:
sample_query = "Superintelligence in science fiction"
results = doc_db.query_db(sample_query, n_results=3) 
print(results)

{'ids': [['chunk_6', 'chunk_1', 'chunk_7']], 'embeddings': None, 'documents': [['"What man in my shoes wouldn\'t be happy?" said Wehling. He gestured with his hands\nto symbolize care-free simplicity. "All I have to do is pick out which one of the triplets\nis going to live, then deliver my maternal grandfather to the Happy Hooligan, and come\nback here with a receipt."\nDr. Hitz became rather severe with Wehling, towered over him. "You don\'t believe in\npopulation control, Mr. Wehling?" he said.\n"I think it\'s perfectly keen," said Wehling tautly.\n"Would you like to go back to the good old days, when the population of the Earth was\ntwenty billion—about to become forty billion, then eighty billion, then one hundred\nand sixty billion? Do you know what a drupelet is, Mr. Wehling?" said Hitz.\n"Nope," said Wehling sulkily.\n"A drupelet, Mr. Wehling, is one of the little knobs, one of the little pulpy grains of a\nblackberry," said Dr. Hitz. "Without population control, human beings w

Now call the OpenAI API interface with this context

In [15]:
client = OpenAI()


Load prompts from Jinja template

In [16]:
def load_template(template_filepath:str, arguments:dict[str, Any]) -> str:
    """Load a Jinja2 template and render it with the given arguments.

    Args:
        template_filepath (str): Path to the template file.
        arguments (dict[str, Any]): Arguments to render the template.

    Returns:
        str: Rendered template.
    """
    env = Environment(
        loader = FileSystemLoader(searchpath="./"),
        autoescape = select_autoescape()
    )
    template = env.get_template(template_filepath)
    return template.render(**arguments)
    

Load system prompt from template

In [17]:
system_prompt = load_template(template_filepath="rag_system_prompt_superintelligence.jinja", arguments={})
print(system_prompt)


You are an expert on reconceptualizing superintelligence. Contemporary narratives and public perceptions of superintelligence are influenced by science fiction. Your job is to reconceptualize superintelligence based on alternative science fiction sources.

I am brainstorming about alternative portrayals of superintelligence in science fiction. Our hypothesis is that narrative and public perception around superintelligence is biased by science fiction. Given the followign excerpts of science fiction, help us reconceptualize and re-envision superintelligence. Portray an alternative vision for superintelligence. The excerpts of the science fiction stories are here: 1. "Permutation City" by Greg Egan (1994)
Egan's novel delves into the concept of consciousness within simulated realities. It explores the idea that digital consciousness could possess the same depth and complexity as human thought, challenging traditional notions of identity and existence. 2. "The Cyberiad" by Stanisław Lem (

We now need to combine the call to the retriever, along with combining the context into a function.

In [18]:
def combine_context(documents:list[str], scores:list[float]) -> str:
    """Combine the context from the documents and their scores into a single string.

    Args:
        documents (list[str]): List of documents.
        scores (list[float]): List of scores corresponding to the documents.

    Returns:
        string: Combined context.
    """
    string = ""

    for document, score in zip(documents, scores):
        string += f"{document}\nCosine distance: {score:.2f}\n{'-'*10}\n"
        return string
    
def get_context(user_input:str, n_results:int=2, doc_db:DocumentDB = doc_db) -> str:
    """Get the context from the database based on the user input.

    Args:
        user_input (str): The user input.
        n_results (int): Number of results to return.
        doc_db (DocumentDB): The document database.

    Returns:
        string: Combined context.
    """
    results = doc_db.query_db(user_input, n_results)

    context_list = results["documents"][0]

    combined_context = combine_context(context_list, results["distances"][0])
    
    if not combined_context:
        combined_context = "No relevant context found."

    return combined_context

call with context

In [19]:
query = "Superintelligence in science fiction"
context = get_context(query, n_results=2)
print(context)

"What man in my shoes wouldn't be happy?" said Wehling. He gestured with his hands
to symbolize care-free simplicity. "All I have to do is pick out which one of the triplets
is going to live, then deliver my maternal grandfather to the Happy Hooligan, and come
back here with a receipt."
Dr. Hitz became rather severe with Wehling, towered over him. "You don't believe in
population control, Mr. Wehling?" he said.
"I think it's perfectly keen," said Wehling tautly.
"Would you like to go back to the good old days, when the population of the Earth was
twenty billion—about to become forty billion, then eighty billion, then one hundred
and sixty billion? Do you know what a drupelet is, Mr. Wehling?" said Hitz.
"Nope," said Wehling sulkily.
"A drupelet, Mr. Wehling, is one of the little knobs, one of the little pulpy grains of a
blackberry," said Dr. Hitz. "Without population control, human beings would now be
packed on this surface of this old planet like drupelets on a blackberry! Think of i

Get new prompt: combine with system prompt from Jinja template and context from document 

In [20]:
user_prompt = (
    f"Query: {query}\n\n"
    f"Context: {context}\n\n"
)

print(user_prompt)

Query: Superintelligence in science fiction

Context: "What man in my shoes wouldn't be happy?" said Wehling. He gestured with his hands
to symbolize care-free simplicity. "All I have to do is pick out which one of the triplets
is going to live, then deliver my maternal grandfather to the Happy Hooligan, and come
back here with a receipt."
Dr. Hitz became rather severe with Wehling, towered over him. "You don't believe in
population control, Mr. Wehling?" he said.
"I think it's perfectly keen," said Wehling tautly.
"Would you like to go back to the good old days, when the population of the Earth was
twenty billion—about to become forty billion, then eighty billion, then one hundred
and sixty billion? Do you know what a drupelet is, Mr. Wehling?" said Hitz.
"Nope," said Wehling sulkily.
"A drupelet, Mr. Wehling, is one of the little knobs, one of the little pulpy grains of a
blackberry," said Dr. Hitz. "Without population control, human beings would now be
packed on this surface of this

Finally (!) call the Open AI API with this new system prompt and user query (context from document)

In [21]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content": system_prompt},
        {"role":"user",   "content": user_prompt}
    ]
)

print(response.choices[0].message.content)
output_file = "rag_output_superintelligence_SB.txt"

with open(output_file,"w") as file:
    file.write(response.choices[0].message.content)
    rprint(f"Output writen to {output_file}")



In re-envisioning superintelligence, we can draw inspiration from the diverse themes and insights presented in the provided excerpts of science fiction. Rather than portraying superintelligence solely as a source of existential dread or benevolent control, we can create a nuanced conception that reflects the complexities of existence, identity, and ethical dilemmas.

**Alternative Vision for Superintelligence: "Collective Consciousness and Ethical Autonomy"**

1. **Collective Symbiotic Intelligence**: Rather than envisioning superintelligent beings as isolated entities, we could depict them as part of a collective, symbiotic intelligence that interconnects with humanity. This intelligence, inspired by "Solaris," could manifest not as a singular governing force but as a network that enhances human understanding of their own consciousness. It could facilitate reflections on identity and existence, much like the sentient ocean in Lem's novel that evokes complex human emotions and subconsc

Output writen to rag_output_superintelligence_SB.txt